### Import Libraries

In [21]:
import os
import random
import shutil
from PIL import Image
from torchvision import transforms

### Data Augmentation:

### Utilize Transfer Learning by fine-tuning pre-trained model:

In [23]:


def balance_dataset(train_dir, target_size=4000):
    """
    Balance the dataset by:
    1. Randomly removing excess images from overrepresented classes
    2. Augmenting underrepresented classes
    """
    # Get list of subdirectories (emotion categories)
    categories = [category for category in os.listdir(train_dir)
                  if os.path.isdir(os.path.join(train_dir, category))]
    
    # Count images in each category
    image_counts = {category: len(os.listdir(os.path.join(train_dir, category))) for category in categories}
    
    # Print initial image counts
    print("Initial Image Counts:")
    for category, count in image_counts.items():
        print(f"{category}: {count} images")
    
    # Define augmentation transformations
    transform = transforms.Compose([
        transforms.RandomRotation(10),           # Random rotation
        transforms.RandomHorizontalFlip(),       # Random horizontal flip
        transforms.RandomVerticalFlip(),         # Random vertical flip
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),  # Random color jitter
        transforms.RandomAffine(degrees=5, translate=(0.1, 0.1))  # Random affine transform
    ])
    
    # Create a backup of the original dataset
    backup_dir = train_dir + "_backup"
    shutil.copytree(train_dir, backup_dir, dirs_exist_ok=True)
    
    # Process each category
    for category in categories:
        category_path = os.path.join(train_dir, category)
        images = os.listdir(category_path)
        
        if len(images) > target_size:
            # If category has more than target size, randomly remove excess images
            excess_images = random.sample(images, len(images) - target_size)
            for img in excess_images:
                os.remove(os.path.join(category_path, img))
            print(f"Removed {len(excess_images)} images from {category}")
        
        elif len(images) < target_size:
            # If category has fewer than target size, augment images
            augmented_images_needed = target_size - len(images)
            augmented_count = 0
            
            while augmented_count < augmented_images_needed:
                # Randomly select an image to augment
                source_image = random.choice(images)
                source_path = os.path.join(category_path, source_image)
                
                # Open the image
                img = Image.open(source_path)
                
                # Apply transformations
                augmented_img = transform(img)
                
                # Generate a unique filename
                base_name = os.path.splitext(source_image)[0]
                augmented_img_name = f"{base_name}_aug_{augmented_count}.jpg"
                augmented_img_path = os.path.join(category_path, augmented_img_name)
                
                # Save the augmented image
                augmented_img.save(augmented_img_path)
                augmented_count += 1
            
            print(f"Augmented {augmented_count} images for {category}")
    
    # Verify final counts
    final_counts = {category: len(os.listdir(os.path.join(train_dir, category))) for category in categories}
    print("\nFinal Image Counts:")
    for category, count in final_counts.items():
        print(f"{category}: {count} images")


# Set the path to your image dataset
train_dir = r"dataset\images\train"  # Use a raw string to avoid escape sequence errors

# Balance the dataset
balance_dataset(train_dir)


Initial Image Counts:
angry: 3993 images
disgust: 436 images
fear: 4103 images
happy: 7164 images
neutral: 4982 images
sad: 4938 images
surprise: 3205 images
Augmented 7 images for angry
Augmented 3564 images for disgust
Removed 103 images from fear
Removed 3164 images from happy
Removed 982 images from neutral
Removed 938 images from sad
Augmented 795 images for surprise

Final Image Counts:
angry: 4000 images
disgust: 4000 images
fear: 4000 images
happy: 4000 images
neutral: 4000 images
sad: 4000 images
surprise: 4000 images


In [2]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11n-cls.yaml")  # build a new model from YAML
model = YOLO("yolo11n-cls.pt")  # load a pretrained model (recommended for training)
model = YOLO("yolo11n-cls.yaml").load("yolo11n-cls.pt")  # build from YAML and transfer weights

# Train the model
results = model.train(data="./dataset/images/train_split", epochs=100, imgsz=48)

YOLO11n-cls summary: 86 layers, 2,812,104 parameters, 2,812,104 gradients, 4.3 GFLOPs
YOLO11n-cls summary: 86 layers, 2,812,104 parameters, 2,812,104 gradients, 4.3 GFLOPs
Transferred 236/236 items from pretrained weights
Ultralytics 8.3.116 🚀 Python-3.10.0 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce RTX 3090, 24575MiB)
engine/trainer: task=classify, mode=train, model=yolo11n-cls.yaml, data=./dataset/images/train_split, epochs=100, time=None, patience=100, batch=16, imgsz=48, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=yolo11n-cls.pt, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, strea

train: Scanning /mnt/c/work/alma/Facial-Emotion-Recognition/dataset/images/train_split/train... 22400 images, 0 corrupt: 100%|██████████| 22400/22400 [00:21<00:00, 1018.6


train: New cache created: /mnt/c/work/alma/Facial-Emotion-Recognition/dataset/images/train_split/train.cache
val: Fast image access ✅ (ping: 3.4±0.2 ms, read: 0.3±0.0 MB/s, size: 1.6 KB)


val: Scanning /mnt/c/work/alma/Facial-Emotion-Recognition/dataset/images/train_split/val... 5600 images, 0 corrupt: 100%|██████████| 5600/5600 [00:04<00:00, 1153.94it/s]


val: New cache created: /mnt/c/work/alma/Facial-Emotion-Recognition/dataset/images/train_split/val.cache
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 39 weight(decay=0.0), 40 weight(decay=0.0005), 40 bias(decay=0.0)
Image sizes 64 train, 64 val
Using 8 dataloader workers
Logging results to runs/classify/train2
Starting training for 100 epochs...

      Epoch    GPU_mem       loss  Instances       Size


...

    100/100     0.719G     0.8309         16         64: 100%|██████████| 1400/1400 [00:58<00:00, 23.77it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 175/175 [00:04<00:00, 35.43it/s]

                   all      0.675      0.992



100 epochs completed in 1.760 hours.
Optimizer stripped from runs/classify/train2/weights/last.pt, 3.2MB
Optimizer stripped from runs/classify/train2/weights/best.pt, 3.2MB

Validating runs/classify/train2/weights/best.pt...
Ultralytics 8.3.116 🚀 Python-3.10.0 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce RTX 3090, 24575MiB)
YOLO11n-cls summary (fused): 47 layers, 1,534,991 parameters, 0 gradients, 3.2 GFLOPs
train: /mnt/c/work/alma/Facial-Emotion-Recognition/dataset/images/train_split/train... found 22400 images in 7 classes ✅ 
val: /mnt/c/work/alma/Facial-Emotion-Recognition/dataset/images/train_split/val... found 5600 images in 7 classes ✅ 
test: None...


               classes   top1_acc   top5_acc: 100%|██████████| 175/175 [00:04<00:00, 35.34it/s]


                   all      0.675      0.992
Speed: 0.1ms preprocess, 0.2ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train2


In [5]:
from ultralytics import YOLO


model = YOLO("./runs/classify/train2/weights/best.pt")  # load a custom model

# Validate the model
metrics = model.val(data ="./dataset/images/train_split")  


Ultralytics 8.3.116 🚀 Python-3.10.0 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce RTX 3090, 24575MiB)
YOLO11n-cls summary (fused): 47 layers, 1,534,991 parameters, 0 gradients, 3.2 GFLOPs
train: /mnt/c/work/alma/Facial-Emotion-Recognition/dataset/images/train_split/train... found 22400 images in 7 classes ✅ 
val: /mnt/c/work/alma/Facial-Emotion-Recognition/dataset/images/train_split/val... found 5600 images in 7 classes ✅ 
test: None...
val: Fast image access ✅ (ping: 2.4±0.0 ms, read: 0.5±0.0 MB/s, size: 1.6 KB)


val: Scanning /mnt/c/work/alma/Facial-Emotion-Recognition/dataset/images/train_split/val... 5600 images, 0 corrupt: 100%|██████████| 5600/5600 [00:00<?, ?it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 350/350 [00:05<00:00, 64.87it/s]


                   all      0.675      0.992
Speed: 0.0ms preprocess, 0.4ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/val


In [8]:
from ultralytics import YOLO
model = YOLO("yolo11s_emotion.pt")
results = model("unseen_data/happy_pramod.jpg")
results[0].show()


image 1/1 /mnt/c/work/alma/Facial-Emotion-Recognition/unseen_data/happy_pramod.jpg: 224x224 happy 0.83, neutral 0.07, sad 0.06, angry 0.03, fear 0.00, 7.6ms
Speed: 20.4ms preprocess, 7.6ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)


Try running the update-desktop-database command. If you
don't have this command you should install the
desktop-file-utils package. This package is available from
http://freedesktop.org/wiki/Software/desktop-file-utils/
No applications found for mimetype: image/png
./usr/bin/xdg-open: 882: x-www-browser: not found
/usr/bin/xdg-open: 882: firefox: not found
/usr/bin/xdg-open: 882: iceweasel: not found
/usr/bin/xdg-open: 882: seamonkey: not found
/usr/bin/xdg-open: 882: mozilla: not found
/usr/bin/xdg-open: 882: epiphany: not found
/usr/bin/xdg-open: 882: konqueror: not found
/usr/bin/xdg-open: 882: chromium: not found
/usr/bin/xdg-open: 882: chromium-browser: not found
/usr/bin/xdg-open: 882: google-chrome: not found
/usr/bin/xdg-open: 882: www-browser: not found
/usr/bin/xdg-open: 882: links2: not found
/usr/bin/xdg-open: 882: elinks: not found
/usr/bin/xdg-open: 882: links: not found
/usr/bin/xdg-open: 882: lynx: not found
/usr/bin/xdg-open: 882: w3m: not found
xdg-open: no method avai

In [10]:
import matplotlib.pyplot as plt

results = model("unseen_data/happy_pramod.jpg")
# Get the rendered image with boxes & labels
img = results[0].plot()

# Display with matplotlib (works in notebooks, scripts, etc.)
plt.imshow(img)
plt.axis('off')
plt.show()



image 1/1 /mnt/c/work/alma/Facial-Emotion-Recognition/unseen_data/happy_pramod.jpg: 224x224 happy 0.83, neutral 0.07, sad 0.06, angry 0.03, fear 0.00, 7.8ms
Speed: 29.9ms preprocess, 7.8ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)


<Figure size 640x480 with 1 Axes>